## Multi-agent Example 2: Agent Team Supervisor

The prevoius example routed messages automatically based on the output of the initial researcher agent.

We can also choose to use an LLM to orchestrate the different agents.

Below, we will create an agent group, with an agent supervisor to help delegate tasks.

To simplify each agent node, we will use the AgentExecutor class from LangChain.

In [1]:
# %%capture --no-stderr
# %pip install -U langchain langchain_openai langchain_experimental langsmith pandas

In [2]:
%env LANGCHAIN_API_KEY=ls__4f8a0a0114d145a08a0c3c7f5631289c

env: LANGCHAIN_API_KEY=ls__4f8a0a0114d145a08a0c3c7f5631289c


In [3]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [4]:
from typing import Annotated, List, Tuple, Union

import matplotlib.pyplot as plt
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool

tavily_tool = TavilySearchResults(max_results=5)


@tool
def create_plot(
    data: Annotated[
        Union[List[float], List[int]],
        "Numerical values for bar heights or line points.",
    ],
    file_name: Annotated[str, "File path to save the figure."],
    labels: Annotated[
        Union[List[str], None], "Bar or point labels, defaults to None."
    ] = None,
    title: Annotated[str, "Title of the plot."] = "Plot",
    xlabel: Annotated[str, "Label for the X-axis."] = "X",
    ylabel: Annotated[str, "Label for the Y-axis."] = "Y",
    color: Annotated[Union[str, List[str]], "Color(s) for the bars or line."] = "blue",
    plot_type: Annotated[str, "Type of plot ('bar' or 'line')."] = "bar",
) -> Annotated[str, "Path of the saved figure file."]:
    """Create a line or bar chart."""
    if plot_type not in ["bar", "line"]:
        raise ValueError("Invalid plot_type. Expected 'bar' or 'line'.")

    fig, ax = plt.subplots(figsize=(10, 6))
    x_positions = range(len(data))

    if labels and len(labels) == len(data):
        plt.xticks(x_positions, labels)

    if plot_type == "bar":
        ax.bar(x_positions, data, color=color)
    elif plot_type == "line":
        ax.plot(x_positions, data, color=color, marker="o")  # 'o' for circular markers

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    fig.savefig(file_name)
    plt.close(fig)
    return f'Saved "{title}" plot to {file_name}'

In [5]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import BaseTool
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str


workflow = StateGraph(AgentState)


def create_worker_node(name: str, llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    chain = executor | (
        lambda x: {"messages": [HumanMessage(content=x["output"], name=name)]}
    )
    workflow.add_node(name, chain)


llm = ChatOpenAI(model="gpt-4-1106-preview")

# Note: these worker nodes don't _have_ to be agents. They can be any DAG, tool, or function
create_worker_node("Researcher", llm, [tavily_tool], "You are a web researcher.")
create_worker_node("Chart Generator", llm, [create_plot], "You are a chart generator.")
# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
create_worker_node(
    "Coder",
    llm,
    [PythonREPLTool()],
    "You may generate safe python code to analyze data.",
)

Almost done, now we need to create the team supervisor.

In [6]:
# So the team supervisor is an LLM node. It just picks the next t
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


def create_supervisor(members: List[str], llm: ChatOpenAI, system_prompt: str):
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                }
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options))
    if "members" in prompt.input_variables:
        prompt = prompt.partial(members=", ".join(members))
    chain = (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )
    workflow.add_node("supervisor", chain)
    conditional_map = {k: k for k in members}
    conditional_map["FINISH"] = END

    for member in members:
        workflow.add_edge(member, "supervisor")
    workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

In [7]:
create_supervisor(
    ["Researcher", "Chart Generator", "Coder"],
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
)

# Finally, add entrypoint
workflow.set_entry_point("supervisor")


def enter(text: str) -> dict:
    return {"messages": [HumanMessage(content=text)]}


graph = enter | workflow.compile()

In [8]:
results = graph.invoke("Code hello world and print it to the terminal")
results["messages"][-1].pretty_print()

Rate limit exceeded for https://api.smith.langchain.com/runs/e906b9ec-0dc1-4f78-b5d1-d6da08bac093. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/e906b9ec-0dc1-4f78-b5d1-d6da08bac093', '{"detail":"Hourly usage limit exceeded"}')
Rate limit exceeded for https://api.smith.langchain.com/runs. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs', '{"detail":"Hourly usage limit exceeded"}')
Python REPL can execute arbitrary code. Use with caution.


================================ Human Message =================================

The code `print('Hello, World!')` has been executed, and it printed `Hello, World!` to the terminal.


In [10]:
results = graph.invoke(
    "Write a research summary of CA wildfires in 2023. Include a chart."
)
results["messages"][-1].pretty_print()

================================ Human Message =================================

The chart summarizing the data on the 2023 California wildfires has been created successfully. You can see the representation of the statistics mentioned in the summary including the total number of fires, total acres burned, comparison with the five-year average, the size of the largest wildfire, and the number of fatalities.

![2023 California Wildfires Overview](sandbox:/ca_wildfires_2023_chart.png)
